In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [2]:

# Directories for the dataset
train_dir = 'skin.disease/skin.disease.split/train'
valid_dir = 'skin.disease/skin.disease.split/valid'
test_dir = 'skin.disease/skin.disease.split/test'



In [3]:
# Parameters
img_size = (224, 224)  # Input image size
batch_size = 32
num_classes = 4  # acne, atopic, bcc, melanoma
epochs = 54  # Increase as necessary



In [4]:
# Data Augmentation and Preprocessing
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
)



In [5]:
valid_datagen = ImageDataGenerator(rescale=1.0 / 255.0)

# Load images from directory
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
)

valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
)

Found 6377 images belonging to 4 classes.
Found 288 images belonging to 4 classes.


In [6]:
# Define the CNN model
model = Sequential()

# First convolutional layer
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_size[0], img_size[1], 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Second convolutional layer
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Third convolutional layer
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Fourth convolutional layer
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten the output from the convolutional layers
model.add(Flatten())

# Fully connected layer
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))  # Dropout for regularization

# Output layer
model.add(Dense(num_classes, activation='softmax'))


C:\Users\azcos5\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [8]:
# Train the model
history = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=epochs,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_steps=valid_generator.samples // batch_size,
)

Epoch 1/54


C:\Users\azcos5\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


199/199 ━━━━━━━━━━━━━━━━━━━━ 150s 727ms/step - accuracy: 0.5137 - loss: 1.0031 - val_accuracy: 0.5417 - val_loss: 0.8362
Epoch 2/54
199/199 ━━━━━━━━━━━━━━━━━━━━ 0s 197us/step - accuracy: 0.5000 - loss: 0.9480  
Epoch 3/54


C:\Users\azcos5\anaconda3\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


199/199 ━━━━━━━━━━━━━━━━━━━━ 115s 565ms/step - accuracy: 0.6241 - loss: 0.7693 - val_accuracy: 0.6458 - val_loss: 0.7350
Epoch 4/54
199/199 ━━━━━━━━━━━━━━━━━━━━ 0s 158us/step - accuracy: 0.7188 - loss: 0.6557  
Epoch 5/54
199/199 ━━━━━━━━━━━━━━━━━━━━ 115s 564ms/step - accuracy: 0.6205 - loss: 0.7781 - val_accuracy: 0.6250 - val_loss: 0.7574
Epoch 6/54
199/199 ━━━━━━━━━━━━━━━━━━━━ 0s 143us/step - accuracy: 0.7188 - loss: 0.5508  
Epoch 7/54
199/199 ━━━━━━━━━━━━━━━━━━━━ 115s 565ms/step - accuracy: 0.6464 - loss: 0.7334 - val_accuracy: 0.6458 - val_loss: 0.7202
Epoch 8/54
199/199 ━━━━━━━━━━━━━━━━━━━━ 0s 143us/step - accuracy: 0.6875 - loss: 0.6932  
Epoch 9/54
199/199 ━━━━━━━━━━━━━━━━━━━━ 114s 562ms/step - accuracy: 0.6575 - loss: 0.7122 - val_accuracy: 0.7361 - val_loss: 0.6864
Epoch 10/54
199/199 ━━━━━━━━━━━━━━━━━━━━ 0s 143us/step - accuracy: 0.7500 - loss: 0.5795  
Epoch 11/54
199/199 ━━━━━━━━━━━━━━━━━━━━ 115s 564ms/step - accuracy: 0.6801 - loss: 0.6819 - val_accuracy: 0.7083 - val_lo

In [9]:
# Print validation accuracy after initial training
print(f"Validation accuracy after initial training: {max(history.history['val_accuracy']) * 100:.2f}%")


Validation accuracy after initial training: 82.99%


In [10]:
 
# Save the model
model.save('skin_disease_classifier_cnn_v3.h5')

In [11]:
# Fine-tuning phase 
# Unfreeze the last few layers of the base model
for layer in model.layers[-20:]:  # Adjust based on your architecture
    layer.trainable = True

# Compile the model again with a lower learning rate
model.compile(optimizer=Adam(learning_rate=0.00001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Continue training with fine-tuning
fine_tune_epochs = 10  # Number of epochs for fine-tuning
history_fine_tune = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=fine_tune_epochs,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_steps=valid_generator.samples // batch_size,
)


Epoch 1/10
199/199 ━━━━━━━━━━━━━━━━━━━━ 118s 568ms/step - accuracy: 0.8512 - loss: 0.3613 - val_accuracy: 0.8194 - val_loss: 0.6076
Epoch 2/10
199/199 ━━━━━━━━━━━━━━━━━━━━ 0s 187us/step - accuracy: 0.9062 - loss: 0.2782  
Epoch 3/10
199/199 ━━━━━━━━━━━━━━━━━━━━ 115s 565ms/step - accuracy: 0.8781 - loss: 0.3155 - val_accuracy: 0.8125 - val_loss: 0.5758
Epoch 4/10
199/199 ━━━━━━━━━━━━━━━━━━━━ 0s 148us/step - accuracy: 0.8750 - loss: 0.3335  
Epoch 5/10
199/199 ━━━━━━━━━━━━━━━━━━━━ 115s 564ms/step - accuracy: 0.8830 - loss: 0.2979 - val_accuracy: 0.8160 - val_loss: 0.5624
Epoch 6/10
199/199 ━━━━━━━━━━━━━━━━━━━━ 0s 138us/step - accuracy: 0.8438 - loss: 0.3335  
Epoch 7/10
199/199 ━━━━━━━━━━━━━━━━━━━━ 115s 563ms/step - accuracy: 0.8792 - loss: 0.3047 - val_accuracy: 0.8056 - val_loss: 0.5559
Epoch 8/10
199/199 ━━━━━━━━━━━━━━━━━━━━ 0s 143us/step - accuracy: 0.7812 - loss: 0.5020  
Epoch 9/10
199/199 ━━━━━━━━━━━━━━━━━━━━ 115s 565ms/step - accuracy: 0.8818 - loss: 0.2939 - val_accuracy: 0.8160

In [12]:
# Print validation accuracy after fine-tuning
print(f"Validation accuracy after fine-tuning: {max(history_fine_tune.history['val_accuracy']) * 100:.2f}%")

Validation accuracy after fine-tuning: 81.94%


In [13]:
# Evaluate on test set
test_generator = valid_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
)

test_loss, test_acc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc:.2f}")


Found 271 images belonging to 4 classes.
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 270ms/step - accuracy: 0.8451 - loss: 0.6399
Test accuracy: 0.82


In [14]:

# Save the model
model.save('skin_disease_classifier_cnn_v3_tuned.h5')